OpenClassrooms
Project 4, Data Scientist
Author : Oumeima EL GHARBI
Date : August 2022

CONTEXTE : Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près à la consommation et aux émissions des bâtiments non destinés à l’habitation.

**OBJECTIF** : vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l’habitation pour lesquels elles n’ont pas encore été mesurées.

NB : Votre prédiction se basera sur les données structurelles des bâtiments (taille et usage des bâtiments, date de construction, situation géographique, ...)


**TO DO** :
En + : Vous cherchez également à évaluer l’intérêt de l’"ENERGY STAR Score" pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe. Vous l'intégrerez dans la modélisation et jugerez de son intérêt.

1) Réaliser une courte analyse exploratoire.
2) Tester différents modèles de prédiction afin de répondre au mieux à la problématique.


**CONSEILS** :

L’objectif est de te passer des relevés de consommation annuels futurs (attention à la fuite de données / DATA LEAKS). Nous ferons de toute façon pour tout nouveau bâtiment un premier relevé de référence la première année, donc rien ne t'interdit d’en déduire des variables structurelles aux bâtiments, par exemple la nature et proportions des sources d’énergie utilisées..

Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).

Mets en place une évaluation rigoureuse des performances de la régression, et optimise les hyperparamètres et le choix d’algorithmes de ML à l’aide d’une validation croisée.

#### Features to predict :

##### Energy Use :

- **SourceEUI(kBtu/sf)** : Source Energy Use Intensity (EUI) is a property's Source Energy Use divided by its gross floor area. Source Energy Use is the annual energy used to operate the property, including losses from generation, transmission, & distribution. Source EUI is measured in thousands of British thermal units (kBtu) per square foot.


- **SiteEnergyUse(kBtu)** : The annual amount of energy consumed by the property from all sources of energy.

- **SteamUse(kBtu)** : The annual amount of district steam consumed by the property on-site, measured in thousands of British thermal units (kBtu).

- **Electricity(kBtu)** : The annual amount of electricity consumed by the property on-site, including electricity purchased from the grid and generated by onsite renewable systems, measured in thousands of British thermal units (kBtu).

- **NaturalGas(kBtu)** : The annual amount of utility-supplied natural gas consumed by the property, measured in thousands of British thermal units (kBtu).

##### Greenhouse Gas Emissions :

- **TotalGHGEmissions** : The total amount of greenhouse gas emissions, including carbon dioxide, methane, and nitrous oxide gases released into the atmosphere as a result of energy consumption at the property, measured in metric tons of carbon dioxide equivalent. This calculation uses a GHG emissions factor from Seattle CIty Light's portfolio of generating resources. This uses Seattle City Light's 2015 emissions factor of 52.44 lbs CO2e/MWh until the 2016 factor is available. Enwave steam factor = 170.17 lbs CO2e/MMBtu. Gas factor sourced from EPA Portfolio Manager = 53.11 kg CO2e/MBtu.


# PART 1 : Cleaning dataset

### Importing libraries

In [679]:
%reset -f

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline
%autosave 300

Autosaving every 300 seconds


In [680]:
from preprocess import *

### 1) Visualizing dataset

#### 1.1) Global statistics

What's in this Dataset?
Rows : 3,376
Columns : 46
Each row is a : Building



source : https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy

#### Loading dataset

In [681]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

# load raw dataset
path = "./dataset/source/"
filename = "2016_Building_Energy_Benchmarking.csv"

dataset_path = "{}{}".format(path, filename)
#all_data = pd.read_csv("dataset/source/2016_Building_Energy_Benchmarking.csv", sep = ',', encoding = 'utf-8')
raw_data = pd.read_csv(dataset_path)

display(raw_data)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.00,0659000030,7,DOWNTOWN,47.61,-122.34,1927,1.00,12,88434,0,88434,Hotel,Hotel,88434.00,NaN,NaN,NaN,NaN,NaN,60.00,81.70,84.30,182.50,189.00,7226362.50,7456910.00,2003882.00,1156514.25,3946027.00,12764.53,1276453.00,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.00,0659000220,7,DOWNTOWN,47.61,-122.33,1996,1.00,11,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.00,Parking,15064.00,Restaurant,4622.00,NaN,61.00,94.80,97.90,176.10,179.40,8387933.00,8664479.00,0.00,950425.19,3242851.00,51450.82,5145082.00,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.00,0659000475,7,DOWNTOWN,47.61,-122.34,1969,1.00,41,956110,196718,759392,Hotel,Hotel,756493.00,NaN,NaN,NaN,NaN,NaN,43.00,96.00,97.70,241.90,244.10,72587024.00,73937112.00,21566554.00,14515435.00,49526664.00,14938.00,1493800.00,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.00,0659000640,7,DOWNTOWN,47.61,-122.34,1926,1.00,10,61320,0,61320,Hotel,Hotel,61320.00,NaN,NaN,NaN,NaN,NaN,56.00,110.80,113.30,216.20,224.00,6794584.00,6946800.50,2214446.25,811525.31,2768924.00,18112.13,1811213.00,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.00,0659000970,7,DOWNTOWN,47.61,-122.34,1980,1.00,18,175580,62000,113580,"Hotel, Parking, Swimming Pool",Hotel,123445.00,Parking,68009.00,Swimming Pool,0.00,NaN,75.00,114.80,118.70,211.40,215.60,14172606.00,14656503.00,0.00,1573448.62,5368607.00,88039.98,8803998.00,False,NaN,Compliant,NaN,505.01,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,Nonresidential COS,Office,Horticulture building,1600 S Dakota St,Seattle,WA,NaN,1624049080,2,GREATER DUWAMISH,47.57,-122.31,1990,1.00,1,12294,0,12294,Office,Office,12294.00,NaN,NaN,NaN,NaN,NaN,46.00,69.10,76.70,161.70,176.10,849745.69,943003.19,0.00,153655.00,524270.86,3254.75,325475.02,True,NaN,Error - Correct Default Data,NaN,20.94,1.70
3372,50223,2016,Nonresidential COS,Other,International district/Chinatown CC,719 8th Ave S,Seattle,WA,NaN,3558300000,2,DOWNTOWN,47.60,-122.32,2004,1.00,1,16000,0,16000,Other - Recreation,Other - Recreation,16000.00,NaN,NaN,NaN,NaN,NaN,NaN,59.40,65.90,114.20,118.90,950276.19,1053705.75,0.00,116221.00,396546.05,5537.30,553729.98,False,NaN,Compliant,NaN,32.17,2.01
3373,50224,2016,Nonresidential COS,Other,Queen Anne Pool,1920 1st Ave W,Seattle,WA,NaN,1794501150,7,MAGNOLIA / QUEEN ANNE,47.64,-122.36,1974,1.00,1,13157,0,13157,"Fitness Center/Health Club/Gym, Other - Recrea...",Other - Recreation,7583.00,Fitness Center/Health Club/Gym,5574.00,Swimming Pool,0.00,NaN,NaN,438.20,460.10,744.80,767.80,5765898.00,6053764.50,0.00,525251.69,1792158.76,39737.39,3973739.06,False,NaN,Compliant,NaN,223.54,16.99
3374,50225,2016,Nonresidential COS,Mixed Use Property,South Park Community Cen

In [682]:
# Primary Key : OSEBuilding ID
raw_data["OSEBuildingID"].unique().shape

(3376,)

In [683]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [684]:
raw_data.describe()

,OSEBuildingID,DataYear,ZipCode,CouncilDistrictCode,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseTypeGFA,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),Comments,TotalGHGEmissions,GHGEmissionsIntensity
count,3376.00,3376.00,3360.00,3376.00,3376.00,3376.00,3376.00,3368.00,3376.00,3376.00,3376.00,3376.00,3356.00,1679.00,596.00,2533.00,3369.00,3370.00,3367.00,3367.00,3371.00,3370.00,3367.00,3367.00,3367.00,3367.00,3367.00,0.00,3367.00,3367.00
mean,21208.99,2016.00,98116.95,4.44,47.62,-122.33,1968.57,1.11,4.71,94833.54,8001.53,86832.01,79177.64,28444.08,11738.68,67.92,54.73,57.03,134.23,137.78,5403667.29,5276725.71,274595.90,1086638.97,3707612.16,13685.05,1368504.54,NaN,119.72,1.18
std,12223.76,0.00,18.62,2.12,0.05,0.03,33.09,2.11,5.49,218837.61,32326.72,207939.81,201703.41,54392.92,29331.20,26.87,56.27,57.16,139.29,139.11,21610628.63,15938786.48,3912173.39,4352478.36,14850656.14,67097.81,6709780.83,NaN,538.83,1.82
min,1.00,2016.00,98006.00,1.00,47.50,-122.41,1900.00,0.00,0.00,11285.00,0.00,3636.00,5656.00,0.00,0.00,1.00,0.00,0.00,0.00,-2.10,0.00,0.00,0.00,-33826.80,-115417.00,0.00,0.00,NaN,-0.80,-0.02
25%,19990.75,2016.00,98105.00,3.00,47.60,-122.35,1948.00,1.00,2.00,28487.00,0.00,27756.00,25094.75,5000.00,2239.00,53.00,27.90,29.40,74.70,78.40,925128.59,970182.23,0.00,187422.95,639487.00,0.00,0.00,NaN,9.50,0.21
50%,23112.00,2016.00,98115.00,4.00,47.62,-122.33,1975.00,1.00,4.00,44175.00,0.00,43216.00,39894.00,10664.00,5043.00,75.00,38.60,40.90,96.20,101.10,1803753.25,1904452.00,0.00,345129.91,1177583.00,3237.54,323754.00,NaN,33.92,0.61
75%,25994.25,2016.00,98122.00,7.00,47.66,-122.32,1997.00,1.00,5.00,90992.00,0.00,84276.25,76200.25,26640.00,10138.75,90.00,60.40,64.28,143.90,148.35,4222455.25,4381429.12,0.00,829317.84,2829632.50,11890.33,1189033.50,NaN,93.94,1.37
max,50226.00,2016.00,98272.00,7.00,47.73,-122.22,2015.00,111.00,99.00,9320156.00,512608.00,9320156.00,9320156.00,686750.00,459748.00,100.00,834.40,834.40,2620.00,2620.00,873923712.00,471613856.00,134943456.00,192577488.00,657074389.00,2979090.00,297909000.00,NaN,16870.98,34.09


#### 1.2) Visualizing building types

In [685]:
all_data = raw_data.copy()

all_building_types = all_data["BuildingType"].unique().tolist()
print(all_building_types, end='\n\n')

all_primary_types = all_data["PrimaryPropertyType"].unique().tolist()
print(all_primary_types, end='\n\n')

all_largest_property_use_type = all_data["LargestPropertyUseType"].unique().tolist()
print(all_largest_property_use_type, end='\n\n')

['NonResidential', 'Nonresidential COS', 'Multifamily MR (5-9)', 'SPS-District K-12', 'Campus', 'Multifamily LR (1-4)', 'Multifamily HR (10+)', 'Nonresidential WA']

['Hotel', 'Other', 'Mid-Rise Multifamily', 'Mixed Use Property', 'K-12 School', 'University', 'Small- and Mid-Sized Office', 'Self-Storage Facility', 'Warehouse', 'Large Office', 'Senior Care Community', 'Medical Office', 'Retail Store', 'Hospital', 'Residence Hall', 'Distribution Center', 'Worship Facility', 'Low-Rise Multifamily', 'Supermarket / Grocery Store', 'Laboratory', 'Refrigerated Warehouse', 'Restaurant', 'High-Rise Multifamily', 'Office']

['Hotel', 'Police Station', 'Other - Entertainment/Public Assembly', 'Multifamily Housing', 'Library', 'Fitness Center/Health Club/Gym', 'Social/Meeting Hall', 'Courthouse', 'Other', 'K-12 School', 'College/University', 'Automobile Dealership', 'Office', 'Self-Storage Facility', 'Non-Refrigerated Warehouse', 'Other - Mall', 'Senior Care Community', 'Medical Office', 'Retail S

In [686]:
residential_BuildingType = ['Multifamily MR (5-9)', 'Multifamily LR (1-4)', 'Multifamily HR (10+)']
residendial_buildings = all_data[all_data["BuildingType"].isin(residential_BuildingType)]

print("If we remove the buildings which BuildingType is 'Multifamily MR (5-9)', 'Multifamily LR (1-4)' or 'Multifamily HR (10+)', then we lose :", residendial_buildings.shape[0], "which is half of our dataset.")

If we remove the buildings which BuildingType is 'Multifamily MR (5-9)', 'Multifamily LR (1-4)' or 'Multifamily HR (10+)', then we lose : 1708 which is half of our dataset.


#### 1.3) Correcting NumberofBuildings

In [687]:
print("There are :", all_data[all_data['NumberofBuildings'] == 0].shape[0], "buildings for which the number of buildings is 0.")
print("The minimum for the number of buildings is 0 which is not possible, we correct that by replacing 0 by 1.")
# Applying the condition
replace_value_for_a_feature(all_data, "NumberofBuildings", 0, 1)

# Verification
all_data[all_data['NumberofBuildings'] == 0]
display(all_data['NumberofBuildings'].describe())

There are : 92 buildings for which the number of buildings is 0.
The minimum for the number of buildings is 0 which is not possible, we correct that by replacing 0 by 1.


count   3368.00
mean       1.13
std        2.10
min        1.00
25%        1.00
50%        1.00
75%        1.00
max      111.00
Name: NumberofBuildings, dtype: float64

#### 1.4) Cleaning Neighborhood

In [688]:
neighborhood_list = all_data["Neighborhood"].unique().tolist()
print(len(neighborhood_list))
print(neighborhood_list)

neighborhood_list_lower = [e.lower() for e in neighborhood_list]
neighborhood_list_lower = list(set(neighborhood_list_lower))

print(len(neighborhood_list_lower))
print(neighborhood_list_lower)

19
['DOWNTOWN', 'SOUTHEAST', 'NORTHEAST', 'EAST', 'Central', 'NORTH', 'MAGNOLIA / QUEEN ANNE', 'LAKE UNION', 'GREATER DUWAMISH', 'BALLARD', 'NORTHWEST', 'CENTRAL', 'SOUTHWEST', 'DELRIDGE', 'Ballard', 'North', 'Delridge', 'Northwest', 'DELRIDGE NEIGHBORHOODS']
14
['delridge neighborhoods', 'northeast', 'magnolia / queen anne', 'northwest', 'delridge', 'southeast', 'ballard', 'downtown', 'lake union', 'greater duwamish', 'east', 'southwest', 'central', 'north']


In [689]:
all_data = map_neighborhoods(all_data)

NameError: name 'map_neighborhoods' is not defined

### 2) Filling ZipCode with k-NN

19 unique neighborhoods, but because of upper case / lower case we get 14 neighborhoods.

The buildings are all located in the same city (Seattle), so we won't use their localisation to predict the Energy Use and GHG Emissions.
We will use CouncilDistrictCode and ZipCode to localize the buildings.

About the localization of the buildings, we will use the ZipCode, however we have 16 missing values, we tried to fill them with a k-NN.

This k-NN below was used for practice purposes. The correct missing ZipCodes were found by searching on internet with the address.


#### 2.1) Preparing ZipCode data

In [ ]:
all_zipcode = all_data["ZipCode"].unique().tolist()
print("We have :", len(all_zipcode), "unique zipcodes.")

# DataFrame with 16 missing ZipCodes
zipcode_na_df = all_data[all_data["ZipCode"].isna()]
print("We have :", zipcode_na_df.shape[0], "missing ZipCodes.")

# We make a list with the address of the building for which the ZipCode is missing.
zipcode_na_list_address = zipcode_na_df["Address"].tolist()
zipcode_na_list = [[i, ""] for i in zipcode_na_list_address]

In [ ]:
# This is the list of zipcodes for each of the 16 missing zipcode. We found it on searching on internet using the Address
correct_zipcode = [98125, 98144, 98117, 98125, 98107, 98117, 98119, 98112, 98122, 98118, 98126, 98108, 98104, 98119, 98108, 98108]
#print(len(right_zipcode))

for i, zipcode in enumerate(correct_zipcode):
    zipcode_na_list[i][1] = zipcode

print(zipcode_na_list)

#### 2.2) ZipCode k-NN

In [ ]:
# K-NN

# 0) data / target
zipcode_not_na = all_data[~all_data["ZipCode"].isna()]

target = zipcode_not_na["ZipCode"]
display(target.shape)

X = zipcode_not_na[["Latitude", "Longitude", "CouncilDistrictCode"]]
display(X.shape)

# 1) Sampling
from sklearn.model_selection import train_test_split
# X = data and y = target
X_train, X_test, y_train, y_test = train_test_split(X, target, train_size=0.8)

In [ ]:
# 2) k-NN and evaluation
from sklearn import neighbors

# Loop on several models ! and check for which value of the hyperparameter k we get the least error percentage.
errors = []
k_start, k_end = 2, 7
for k in range(k_start, k_end):
    knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    # We "fit" the model to our training data and compute the percentage of correct predictions with "score"
    errors.append(100*(1 - knn.fit(X_train, y_train).score(X_test, y_test)))
plt.plot(range(k_start, k_end), errors, 'o-')
plt.show()

In [ ]:
# 3) predict !!
# We use the best classifier to predict
k = 3
knn = neighbors.KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

X_pred = zipcode_na_df[["Latitude", "Longitude", "CouncilDistrictCode"]] # 16 missing postal code
correct_target = np.array(correct_zipcode, dtype=np.float64) # 16 correct ZipCodes / dtype so that it won't be int32

y_pred = knn.predict(X_pred)
print(y_pred)
print(correct_target)

# Accuracy of what X_pred predicted (y_pred) and correct_targer
knn.score(X_pred, correct_target)
# Accuracy 14/16 = 0.875, means two wrong predictions.
# Accuracy 15/16 = 0.9375, means one wrong prediction. ;) with CouncilDistrict we get better results ;)

error = 1 - knn.score(X_pred, correct_target)
print('Error in prediction : %.3f' % error) # .1 means 1 decimal, .2 means 2 decimals
print('Error in prediction : {:.1%}'.format(error))

In [ ]:
print("We replace the missing ZipCodes by their correct value.")

#zipcode_na_df.index
# We iterate on the index of the buildings for which the ZipCode is missing
all_data_v1 = all_data.copy()
for i, index in enumerate(zipcode_na_df.index):
    all_data_v1.at[index, "ZipCode"] = correct_zipcode[i]

# Verification
#all_data_v1["ZipCode"].shape
display(all_data_v1[all_data_v1["ZipCode"].isna()])


### 3) Dropping features

We have 46 columns / features to represent each building.

We select the interesting features that will help us predict the energy consumption of the buildings.

We keep for now **ComplianceStatus** and **Outlier** so that we can remove outlier buildings.

In [ ]:
columns_to_drop = ["DataYear", "PropertyName", "ListOfAllPropertyUseTypes", "Address", "City", "State", "TaxParcelIdentificationNumber", "YearsENERGYSTARCertified", "DefaultData", "Comments", "Latitude", "Longitude"]

print("We remove latitude and longitude so that our algorithm won't have too much features. We will localize the buildings based on the Zip Code and the Council District Code.")

In [ ]:
all_data_v1 = drop_selected_features(all_data_v1, columns_to_drop)
display(all_data_v1)

### 4) Verifying GFA

In [ ]:
# df = all_data[["PropertyGFATotal", "PropertyGFABuilding(s)", "PropertyGFAParking"]]
# df.describe()

verify_PropertyGFA(all_data_v1)

We have checked that the Total Property GFA is equal to the Total Building(s) GFA + Total Parking GFA.

### 5) Verifying Property Use Type

We display the proportion of NaN per feature for our dataset.

**Analysis** :
- We notice with the barplot below that the Second and Third Largest Property (GFA and Type) have more than 50% of missing values.

- Outliers too is a feature with more than 90% of missing values : that's because if the value is missing it means the building is not an outlier.

In [ ]:
# We use head(10) to only display 10 features that have the most missing values.
data_nan = all_data_v1.isna().sum().sort_values(ascending=False).head(10)

#data_nan

In [ ]:
data_nan.values
all_data_v1.shape

In [ ]:
plt.figure(figsize=(10,8))
plt.title('Proportion of NaN per feature (%)')
sns.barplot(x= 100 * data_nan.values / all_data_v1.shape[0], y=data_nan.index)

In [ ]:
all_data_v2 = fill_property_use_type_GFA(all_data_v1)

display(all_data_v2)

We have filled Second and Third Property Use Type GFA by 0 (meaning there are no second or  third use for the building.
We have also filled the Second and Third Property USe Type by "None".

#### Change variable's type

In [ ]:
columns_to_categorize = ["OSEBuildingID", "BuildingType", "PrimaryPropertyType", "Neighborhood", "ZipCode", "CouncilDistrictCode", "YearBuilt", "LargestPropertyUseType", "SecondLargestPropertyUseType", "ThirdLargestPropertyUseType"]

all_data_v2 = assign_type_column(all_data_v2, columns_to_categorize, "category")

print("We have changed the type of the categorical features to 'category'.")
display(all_data_v2.dtypes)

### 6) Outliers cleaning

#### 6.1) 0utliers described in the dataset

Outlier : Whether a property is a high or low outlier

In [ ]:
print("Checking Outliers.")

# We have a column called "Outlier" in which we have 32 buildings which are considered outliers.
outliers_examples = all_data_v2[~all_data_v2["Outlier"].isna()] # 32 outliers in raw dataset and in all_data_v2

display(outliers_examples)

#### 6.2) Compliance Status described in the dataset


In [ ]:
print("This is an histogram that presents the distribution of the values of the variable ComplianceStatus in %.")
sns.displot(data=all_data_v2, x="ComplianceStatus", stat='percent', height=8, aspect=1) # aspect : to make it wider

In [ ]:
print("This is an histogram that presents the distribution of the values of the variable ComplianceStatus by counting them.")
# to make the graphs bigger
sns.set(rc={'figure.figsize':(12,8)})
sns.countplot(data=all_data_v2, x="ComplianceStatus") # countnplot is for discrete variable / categories here.

In [ ]:
print("Checking ComplianceStatus.")
display(all_data_v2["ComplianceStatus"].unique())

non_compliant_status = ['Error - Correct Default Data', 'Missing Data', 'Non-Compliant']

non_compliant_buildings = all_data_v2[all_data_v2["ComplianceStatus"].isin(non_compliant_status)]
display(non_compliant_buildings)

print("In the raw dataset, we had :", all_data[all_data["ComplianceStatus"].isin(non_compliant_status)].shape[0], "non compliant buildings.")

In [ ]:
all_data_v3 = drop_outliers_based_on_dataset(all_data_v2)

In [ ]:
print("We have removed 60 buildings which were not compliant. And amongst these 60 buildings, 32 were outliers.")

#### 6.3) Cleaning NaN values

##### 6.3.1) Removing buildings with NaN values

In [ ]:
columns_with_nan = all_data_v3.columns[all_data_v3.isna().any()].tolist()
print("These columns have empty values (NaN) :", columns_with_nan, end='\n\n')
print("For the Energy Star Score, we will keep it untouched until later.", end='\n\n')

x = all_data_v3[all_data_v3["SiteEUIWN(kBtu/sf)"].isna()]
display(x)

y = all_data_v3[all_data_v3["SiteEnergyUseWN(kBtu)"].isna()]
display(y)

In [ ]:
print("We remove 1 building with SiteEUI(kBtu/sf) and SiteEnergyUseWN(kBtu) as NaN.")

features_with_nan = ["SiteEUI(kBtu/sf)", "SiteEUIWN(kBtu/sf)"]
all_data_v4 = drop_buildings_subset_nan(all_data_v3, features_with_nan)

##### 6.3.2) Cleaning Energy Star Score

In [ ]:
print("ENERGYSTARScore and its NaN :")

print("For the", all_data_v4[all_data_v4["ENERGYSTARScore"].isna()].shape[0], "buildings that do not have an Energy Star Score, we fill it with -1 for now.")
#all_data_v4[all_data_v4["ENERGYSTARScore"].isna()].count

all_data_v4 = fill_nan_column_by_value(all_data_v4, "ENERGYSTARScore", -1)

#### 6.4) Energy or GHG Outliers (negative values)

In [ ]:
display(all_data_v4.describe())
print(all_data_v4.shape)


In [ ]:
print("We can see that we have negative values for theses features : ")

all_energy_features = ["SiteEUI(kBtu/sf)", "SiteEUIWN(kBtu/sf)", "SourceEUI(kBtu/sf)", "SourceEUIWN(kBtu/sf)", "SiteEnergyUse(kBtu)", "SiteEnergyUseWN(kBtu)", "SteamUse(kBtu)", "Electricity(kWh)", "Electricity(kBtu)", "NaturalGas(therms)", "NaturalGas(kBtu)", "TotalGHGEmissions", "GHGEmissionsIntensity"]

features_with_negative_values = verify_min_value(all_data_v4, all_energy_features, 0)

print("This is the list of features for which we will remove outliers :", features_with_negative_values)

In [ ]:
all_data_v5 = removing_outliers(all_data_v4, "SourceEUIWN(kBtu/sf)", 0, less_than_or_equal=False)
print("We had one outlier building for which the energy values were negative.")

display(all_data_v5.describe())
print(all_data_v5.shape)

In [ ]:
print("We run the same program to check if we still have negative values for the energy.")
verify_min_value(all_data_v5, all_energy_features, 0)

### 7) Verification Total Energy

We consider that the **Total Energy** of a building is represented by the variable **SiteEnergyUse(kBtu)**.
Thus  **SiteEnergyUse(kBtu)** = **Electricity(kBtu)** + **SteamUse(kBtu)** + **NaturalGas(kBtu)** + **OtherEnergy(kBtu)**

We will compute the difference between the Total Energy and Electricity(kBtu) + SteamUse(kBtu) + NaturalGas(kBtu) and if the difference / error / other energy is greater than 1% we drop the building.

In [ ]:
print("The computed remaining energy is the absolute value of the percentage of total energy that remaining energy has.")
print("The computation of the remaining energy is based on a hypothesis that the site energy is the sum of electricity, steam and natural gas.")
all_data_v6 = compute_TotalEnergy(all_data_v5).sort_values(by="RemainingEnergy(%)", ascending=False)
display(all_data_v6)

In [ ]:
all_data_v7 = removing_outliers(all_data_v6, "RemainingEnergy(%)", 0.01, less_than_or_equal=True)
display(all_data_v7)

In [ ]:
total_energy_outliers = all_data_v6[all_data_v6["RemainingEnergy(%)"] > 0.01]
display(total_energy_outliers)

print("By removing these :", total_energy_outliers.shape[0], "outlier buildings, we lose :", round(100 * total_energy_outliers.shape[0] / all_data_v6.shape[0], 2), "% of our buildings.")

In [ ]:
print("We remove the two columns about the remaining energy")
all_data_v8 = all_data_v7.drop(columns=["RemainingEnergy(kBtu)", "RemainingEnergy(%)"])
all_data_v8.shape

print("ASK MENTOR")

### 8) Final step

#### 8.1) Boxplot and Outliers

In [ ]:
from functions import *

print("After removing outliers, we display boxplot per energy feature and per Primary Property Type.")
print("We suppose that a K-12 School has different energy needed compared to a hospital, so the boxplot must represent that difference.")

display_boxplot_per_feature(all_data_v8, all_energy_features, "PrimaryPropertyType")

data = all_data_v7.copy()
data.shape

print("We could see outliers with the boxplot, we count here the number of outliers per energy feature.")


total_buildings = all_data_v7.shape[0]
print(total_buildings)

for column in all_energy_features:
    "outliers aux extrémités des distributions"
    print(column)
    data.loc[data[column] > data[column].quantile(0.995)] = np.nan
    data.loc[data[column] < data[column].quantile(0.005)] = np.nan

print(data.shape)

data.info()

for column in all_energy_features:
    print('\n',column)
    print('Percentage of NaN : ', round(data[column].isna().sum() / data[column].shape[0] * 100), '%')
    print('Uniques : ', data[column].nunique())

#### 8.2) Outliers / Percentile 0.5% and Percentile 99.5%

##### Checking upper outliers and lower outliers

##### We remove the percentile  95 and percentile 5.
We remove the outliers which values for the Total Energy greater than the percentile 95 and lower than the percentile 5.

In [ ]:
print("Before")
all_data_v8.info()

In [ ]:
print("After")
all_data_v9 = remove_percentile_outliers(all_data_v8, 0.995, 0.005, ["SiteEnergyUse(kBtu)", "TotalGHGEmissions"])
display(all_data_v9.info())

#### 8.3) Saving final cleaned dataset

In [ ]:
# We reset the index
all_data_vf = all_data_v9.reset_index(drop = True)

# Save
export_path = "./dataset/cleaned/"
export_filename = "2016_Building_Energy_Cleaned.csv"

all_data_vf.to_csv("{}{}".format(export_path, export_filename), index=False)